In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
with open('sample.json', 'r') as file:
    data = json.load(file)

In [6]:
column_names = ['date', 'hasGoal', 'hasRevenue', 'sessionCount', 'sessions', 'deviceCategory',
                'activitySummary', 'channel','type', 'repeatActivityTimes', 'details', 'Page URL', 'pageTitle',
                'time']

In [7]:
data_values = []
for entry in data['dates']:
    row_values = []
    for column_name in column_names:
        if column_name in entry:
            row_values.append(entry[column_name])
        else:
            row_values.append(None)
    data_values.append(row_values)

In [8]:
df = pd.DataFrame(data_values, columns=column_names)

In [9]:
df

,date,hasGoal,hasRevenue,sessionCount,sessions,deviceCategory,activitySummary,channel,type,repeatActivityTimes,details,Page URL,pageTitle,time
0,"Jun 28, 2023",False,False,8,"[{'duration': '00:37', 'deviceCategory': 'desk...",None,None,None,None,None,None,None,None,None
1,"Jun 27, 2023",False,False,5,"[{'duration': '25:56', 'deviceCategory': 'desk...",None,None,None,None,None,None,None,None,None
2,"Jun 26, 2023",False,False,5,"[{'duration': '12:05', 'deviceCategory': 'desk...",None,None,None,None,None,None,None,None,None
3,"Jun 25, 2023",False,False,4,"[{'duration': '01:30', 'deviceCategory': 'desk...",None,None,None,None,None,None,None,None,None
4,"Jun 24, 2023",False,False,3,"[{'duration': '00:00', 'deviceCategory': 'desk...",None,None,None,None,None,None,None,None,None


In [10]:
# Extract information from 'sessions' column
sessions_data = []
for entry in data['dates']:
    for session in entry['sessions']:
        session_info = {
            'date': entry['date'],
            'hasGoal': entry['hasGoal'],
            'hasRevenue': entry['hasRevenue'],
            'sessionCount': entry['sessionCount'],
            'deviceCategory': session['deviceCategory'],
            'channel': session['channel'],
            'type': session['activities'][0]['type'] if session['activities'] else None,
            'repeatActivityTimes': session['activities'][0]['repeatActivityTimes'] if session['activities'] else None,
            'details': session['activities'][0]['details'][0] if session['activities'] else None,
            'pageTitle': session['activities'][0]['pageTitle'] if session['activities'] else None,
            'pageURL': session['activities'][0]['details'][0]['Page URL'][0] if session['activities'] else None,
            'time': session['activities'][0]['time'] if session['activities'] else None
        }
        sessions_data.append(session_info)

# Create DataFrame from extracted data
NewDF = pd.DataFrame(sessions_data)

In [11]:
NewDF.head(3)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,details,pageTitle,pageURL,time
0,"Jun 28, 2023",False,False,8,desktop,Referral,PAGEVIEW,[],{'Page title': ['Why is My Website Not Loading...,Why is My Website Not Loading Correctly in the...,/hc/en-us/articles/360021311333-Why-is-My-Webs...,10:40 PM
1,"Jun 28, 2023",False,False,8,desktop,Referral,PAGEVIEW,[],{'Page title': ['Working With Metrics in VWO â...,Working With Metrics in VWO â€“ VWO,/hc/en-us/articles/8675547113625-Working-With-...,8:37 PM
2,"Jun 28, 2023",False,False,8,desktop,Referral,PAGEVIEW,[],{'Page title': ['Getting Started with VWO â€“ ...,Getting Started with VWO â€“ VWO,/hc/en-us/categories/360001529093,7:04 PM


In [12]:
print(f'Total Rows:> {NewDF.shape[0]}')
print(f'Total Columns:> {NewDF.shape[1]}')

Total Rows:> 25
Total Columns:> 12


In [13]:
#Checking Null values 
NewDF.isnull().sum()

date                   0
hasGoal                0
hasRevenue             0
sessionCount           0
deviceCategory         0
channel                0
type                   0
repeatActivityTimes    0
details                0
pageTitle              0
pageURL                0
time                   0
dtype: int64

In [14]:
NewDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 25 non-null     object
 1   hasGoal              25 non-null     bool  
 2   hasRevenue           25 non-null     bool  
 3   sessionCount         25 non-null     int64 
 4   deviceCategory       25 non-null     object
 5   channel              25 non-null     object
 6   type                 25 non-null     object
 7   repeatActivityTimes  25 non-null     object
 8   details              25 non-null     object
 9   pageTitle            25 non-null     object
 10  pageURL              25 non-null     object
 11  time                 25 non-null     object
dtypes: bool(2), int64(1), object(9)
memory usage: 2.1+ KB


In [15]:
#changing date to datetime formate
NewDF['date'] = pd.to_datetime(NewDF['date'])

In [16]:
# Convert "time" column to datetime format
NewDF['time'] = pd.to_datetime(NewDF['time'])
# Extract time component from datetime column
NewDF['time'] = NewDF['time'].dt.time

In [17]:
NewDF.head(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,details,pageTitle,pageURL,time
0,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],{'Page title': ['Why is My Website Not Loading...,Why is My Website Not Loading Correctly in the...,/hc/en-us/articles/360021311333-Why-is-My-Webs...,22:40:00
1,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],{'Page title': ['Working With Metrics in VWO â...,Working With Metrics in VWO â€“ VWO,/hc/en-us/articles/8675547113625-Working-With-...,20:37:00


In [18]:
NewDF['details'].value_counts()

{'Page title': ['Configuring the VWO SmartCode for Your Website â€“ VWO'], 'Page URL': ['/hc/en-us/articles/360019422834-Configuring-the-VWO-SmartCode-for-Your-Website']}                                                   3
{'Page title': ['Integrating VWO With Google Analytics 4 (GA4) â€“ VWO'], 'Page URL': ['/hc/en-us/articles/17441211466009-Integrating-VWO-With-Google-Analytics-4-GA4-']}                                                    3
{'Page title': ['Using URL Patterns in VWO â€“ VWO'], 'Page URL': ['/hc/en-us/articles/360021127714-Using-URL-Patterns-in-VWO']}                                                                                             3
{'Page title': ['Configuring Single Page Application / Server Side Rendering Support in Editor Settings for Your Website in VWO â€“ VWO'], 'Page URL': ['/hc/en-us/articles/900000678806']}                                  2
{'Page title': ['Working With Metrics in VWO â€“ VWO'], 'Page URL': ['/hc/en-us/articles/8675547113625-Worki

In [19]:
# Dropping the 'details' column
NewDF.drop('details', axis=1, inplace=True)

In [20]:
NewDF.head(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,pageTitle,pageURL,time
0,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],Why is My Website Not Loading Correctly in the...,/hc/en-us/articles/360021311333-Why-is-My-Webs...,22:40:00
1,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],Working With Metrics in VWO â€“ VWO,/hc/en-us/articles/8675547113625-Working-With-...,20:37:00


In [21]:
NewDF['pageURL']

0     /hc/en-us/articles/360021311333-Why-is-My-Webs...
1     /hc/en-us/articles/8675547113625-Working-With-...
2                     /hc/en-us/categories/360001529093
3     /hc/en-us/articles/360019422834-Configuring-th...
4     /hc/en-us/articles/17441211466009-Integrating-...
5     /hc/en-us/articles/360019593433-Adding-Users-a...
6     /hc/en-us/articles/360021127714-Using-URL-Patt...
7     /hc/en-us/articles/360019420774-What-is-VWO-Sm...
8     /hc/en-us/articles/17441211466009-Integrating-...
9                       /hc/en-us/articles/360033597014
10    /hc/en-us/articles/360019422834-Configuring-th...
11                      /hc/en-us/articles/360021310593
12    /hc/en-us/articles/360019422834-Configuring-th...
13    /hc/en-us/articles/360021127714-Using-URL-Patt...
14    /hc/en-us/articles/360019594593-How-to-Change-...
15    /hc/en-us/articles/4402037837209-How-to-Whitel...
16                      /hc/en-us/articles/360021465793
17                      /hc/en-us/articles/90000

In [22]:
# Extract domain name from 'Page URL'
NewDF['domain'] = NewDF['pageURL'].str.split('/').str[2]

# Extract keywords from 'pageTitle'
NewDF['keywords'] = NewDF['pageTitle'].str.lower().str.split()

In [23]:
# Extract hour of the day from 'time'
NewDF['hour'] = NewDF['time'].apply(lambda x: x.hour if x is not None else None)

In [24]:
NewDF.sample(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,pageTitle,pageURL,time,domain,keywords,hour
24,2023-06-24,False,False,3,desktop,Referral,PAGEVIEW,[],Troubleshooting: GA Integration Not Working â€...,/hc/en-us/articles/360033967194,17:58:00,en-us,"[troubleshooting:, ga, integration, not, worki...",17
6,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],Using URL Patterns in VWO â€“ VWO,/hc/en-us/articles/360021127714-Using-URL-Patt...,14:33:00,en-us,"[using, url, patterns, in, vwo, â€“, vwo]",14


In [25]:
NewDF['pageURL'].value_counts()

/hc/en-us/articles/360019422834-Configuring-the-VWO-SmartCode-for-Your-Website                          3
/hc/en-us/articles/17441211466009-Integrating-VWO-With-Google-Analytics-4-GA4-                          3
/hc/en-us/articles/360021127714-Using-URL-Patterns-in-VWO                                               3
/hc/en-us/articles/900000678806                                                                         2
/hc/en-us/articles/8675547113625-Working-With-Metrics-in-VWO                                            2
/hc/en-us/articles/360021311333-Why-is-My-Website-Not-Loading-Correctly-in-the-VWO-Visual-Editor-       1
/hc/en-us/articles/4402037837209-How-to-Whitelist-VWO-in-Your-Website-s-Content-Security-Policy-CSP-    1
/hc/en-us/articles/360033967194-Troubleshooting-GA-Integration-Not-Working                              1
/hc/en-us/articles/360021124994-Troubleshooting-Variation-Changes-Are-Not-Appearing-in-VWO              1
/hc/en-us/articles/360021465793               

In [26]:
NewDF['DayOfWeek'] = pd.to_datetime(NewDF['date']).dt.day_name()

In [27]:
import re
NewDF['article_id'] = NewDF['pageURL'].str.extract(r'/articles/(\d+)')
NewDF['category'] = NewDF['pageURL'].str.extract(r'/hc/en-us/articles/(\w+)')
NewDF['is_troubleshooting'] = NewDF['pageURL'].str.contains('troubleshooting', case=False)

In [28]:
NewDF.head(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,pageTitle,pageURL,time,domain,keywords,hour,DayOfWeek,article_id,category,is_troubleshooting
0,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],Why is My Website Not Loading Correctly in the...,/hc/en-us/articles/360021311333-Why-is-My-Webs...,22:40:00,en-us,"[why, is, my, website, not, loading, correctly...",22,Wednesday,360021311333,360021311333,False
1,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],Working With Metrics in VWO â€“ VWO,/hc/en-us/articles/8675547113625-Working-With-...,20:37:00,en-us,"[working, with, metrics, in, vwo, â€“, vwo]",20,Wednesday,8675547113625,8675547113625,False


In [29]:
NewDF['is_troubleshooting'].value_counts()

False    23
True      2
Name: is_troubleshooting, dtype: int64

In [30]:
NewDF.drop(['pageURL', 'pageTitle'], axis=1, inplace=True)

In [31]:
NewDF.sample(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,time,domain,keywords,hour,DayOfWeek,article_id,category,is_troubleshooting
7,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],12:19:00,en-us,"[what, is, vwo, smartcode?, â€“, vwo]",12,Wednesday,360019420774,360019420774,False
18,2023-06-25,False,False,4,desktop,Referral,PAGEVIEW,[],22:05:00,en-us,"[configuring, single, page, application, /, se...",22,Sunday,900000678806,900000678806,False


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [33]:
# Convert list of keywords to string
NewDF['keywords_processed'] = NewDF['keywords'].apply(lambda x: ' '.join(x)) 

In [34]:
# Vectorization using TF-IDF
vectorizer = TfidfVectorizer()
keywords_vectorized = vectorizer.fit_transform(NewDF['keywords_processed'])

In [35]:
# Count of unique keywords
NewDF['keywords_count'] = NewDF['keywords'].apply(lambda x: len(set(x)))

In [36]:
# Dimensionality Reduction
pca = PCA(n_components=2)
keywords_pca = pca.fit_transform(keywords_vectorized.toarray())
NewDF['pca_component_1'] = keywords_pca[:, 0]
NewDF['pca_component_2'] = keywords_pca[:, 1]

In [37]:
# Drop unnecessary columns
NewDF.drop(['keywords', 'keywords_processed'], axis=1, inplace=True)

In [38]:
NewDF.head(2)

,date,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,time,domain,hour,DayOfWeek,article_id,category,is_troubleshooting,keywords_count,pca_component_1,pca_component_2
0,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],22:40:00,en-us,22,Wednesday,360021311333,360021311333,False,13,0.198124,-0.026648
1,2023-06-28,False,False,8,desktop,Referral,PAGEVIEW,[],20:37:00,en-us,20,Wednesday,8675547113625,8675547113625,False,6,-0.267882,0.057769


In [39]:
# Extract binary weekday/weekend indicator
NewDF['IsWeekend'] = NewDF['date'].dt.dayofweek // 5

# Perform one-hot encoding
NewDF = pd.get_dummies(NewDF, columns=['DayOfWeek'])

In [40]:
# Drop the original 'date' column
NewDF = NewDF.drop(['date','time'], axis=1)

In [41]:
NewDF['is_troubleshooting'] = NewDF['is_troubleshooting'].astype(int)
NewDF['hasGoal'] = NewDF['hasGoal'].astype(int)
NewDF['hasRevenue'] = NewDF['hasRevenue'].astype(int)

In [42]:
NewDF.sample(3)

,hasGoal,hasRevenue,sessionCount,deviceCategory,channel,type,repeatActivityTimes,domain,hour,article_id,category,is_troubleshooting,keywords_count,pca_component_1,pca_component_2,IsWeekend,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
3,0,0,8,desktop,Referral,PAGEVIEW,[],en-us,17,360019422834,360019422834,0,8,0.631514,-0.369254,0,0,0,0,0,1
15,0,0,5,desktop,Referral,PAGEVIEW,[],en-us,15,4402037837209,4402037837209,0,12,0.063942,0.076644,0,1,0,0,0,0
7,0,0,8,desktop,Referral,PAGEVIEW,[],en-us,12,360019420774,360019420774,0,5,0.143906,-0.085549,0,0,0,0,0,1


In [43]:
NewDF['domain']

0     en-us
1     en-us
2     en-us
3     en-us
4     en-us
5     en-us
6     en-us
7     en-us
8     en-us
9     en-us
10    en-us
11    en-us
12    en-us
13    en-us
14    en-us
15    en-us
16    en-us
17    en-us
18    en-us
19    en-us
20    en-us
21    en-us
22    en-us
23    en-us
24    en-us
Name: domain, dtype: object

In [42]:
# dropping these ['deviceCategory', 'channel', 'type'], coz this is not much usefull

In [44]:
NewDF = NewDF.drop(['deviceCategory', 'channel', 'type', 'domain','repeatActivityTimes','category'], axis=1)

In [45]:
NewDF

,hasGoal,hasRevenue,sessionCount,hour,article_id,is_troubleshooting,keywords_count,pca_component_1,pca_component_2,IsWeekend,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,0,0,8,22,360021311333,0,13,0.198124,-0.026648,0,0,0,0,0,1
1,0,0,8,20,8675547113625,0,6,-0.267882,0.057769,0,0,0,0,0,1
2,0,0,8,19,NaN,0,5,-0.193420,-0.073698,0,0,0,0,0,1
3,0,0,8,17,360019422834,0,8,0.631514,-0.369254,0,0,0,0,0,1
4,0,0,8,16,17441211466009,0,8,-0.621174,-0.465835,0,0,0,0,0,1
5,0,0,8,15,360019593433,0,9,0.070804,0.058726,0,0,0,0,0,1
6,0,0,8,14,360021127714,0,6,-0.048853,0.698630,0,0,0,0,0,1
7,0,0,8,12,360019420774,0,5,0.143906,-0.085549,0,0,0,0,0,1
8,0,0,5,21,17441211466009,0,8,-0.621174,-0.465835,0,0,0,0,1,0
9,0,0,5,19,360033597014,0,12,-0.003547,0.115438,0,0,0,0,1,0


In [46]:
NewDF = NewDF.dropna()

In [47]:
NewDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 24
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hasGoal              24 non-null     int32  
 1   hasRevenue           24 non-null     int32  
 2   sessionCount         24 non-null     int64  
 3   hour                 24 non-null     int64  
 4   article_id           24 non-null     object 
 5   is_troubleshooting   24 non-null     int32  
 6   keywords_count       24 non-null     int64  
 7   pca_component_1      24 non-null     float64
 8   pca_component_2      24 non-null     float64
 9   IsWeekend            24 non-null     int64  
 10  DayOfWeek_Monday     24 non-null     uint8  
 11  DayOfWeek_Saturday   24 non-null     uint8  
 12  DayOfWeek_Sunday     24 non-null     uint8  
 13  DayOfWeek_Tuesday    24 non-null     uint8  
 14  DayOfWeek_Wednesday  24 non-null     uint8  
dtypes: float64(2), int32(3), int64(4), object(

In [49]:
NewDF['article_id'] = NewDF['article_id'].astype('int64')

In [50]:
NewDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 24
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hasGoal              24 non-null     int32  
 1   hasRevenue           24 non-null     int32  
 2   sessionCount         24 non-null     int64  
 3   hour                 24 non-null     int64  
 4   article_id           24 non-null     int64  
 5   is_troubleshooting   24 non-null     int32  
 6   keywords_count       24 non-null     int64  
 7   pca_component_1      24 non-null     float64
 8   pca_component_2      24 non-null     float64
 9   IsWeekend            24 non-null     int64  
 10  DayOfWeek_Monday     24 non-null     uint8  
 11  DayOfWeek_Saturday   24 non-null     uint8  
 12  DayOfWeek_Sunday     24 non-null     uint8  
 13  DayOfWeek_Tuesday    24 non-null     uint8  
 14  DayOfWeek_Wednesday  24 non-null     uint8  
dtypes: float64(2), int32(3), int64(5), uint8(5

In [51]:
# considering hasGoal is out target column
x = NewDF.drop(['hasGoal'],axis = 1)

In [52]:
y = NewDF['hasGoal']

In [95]:
x,y

(    hasRevenue  sessionCount  hour      article_id  is_troubleshooting  \
 0            0             8    22    360021311333                   0   
 1            0             8    20   8675547113625                   0   
 3            0             8    17    360019422834                   0   
 4            0             8    16  17441211466009                   0   
 5            0             8    15    360019593433                   0   
 6            0             8    14    360021127714                   0   
 7            0             8    12    360019420774                   0   
 8            0             5    21  17441211466009                   0   
 9            0             5    19    360033597014                   0   
 10           0             5    18    360019422834                   0   
 11           0             5    15    360021310593                   0   
 12           0             5    14    360019422834                   0   
 13           0          

In [ ]:
# training the model 

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [55]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((19, 14), (19,), (5, 14), (5,))

In [56]:
#Scaling data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [57]:
XtrainScale = scale.fit_transform(x_train)

In [58]:
XtestScale = scale.transform(x_test)

In [59]:
XtrainScale

array([[ 0.        ,  1.23616309,  0.59655068,  0.81474327, -0.34299717,
        -0.82676738, -0.68080972,  0.24684496, -0.5976143 , -0.51639778,
        -0.34299717, -0.4330127 , -0.4330127 ,  1.30930734],
       [ 0.        , -0.4024717 ,  0.30592343, -0.50747112, -0.34299717,
         0.04593152,  0.03024258,  0.48365991, -0.5976143 ,  1.93649167,
        -0.34299717, -0.4330127 , -0.4330127 , -0.76376262],
       [ 0.        , -1.49489489, -0.27533108, -0.50746884, -0.34299717,
        -0.53586775, -0.2407972 ,  0.53693641,  1.67332005, -0.51639778,
         2.91547595, -0.4330127 , -0.4330127 , -0.76376262],
       [ 0.        , -0.4024717 , -1.72846737, -0.42161118, -0.34299717,
         2.3731286 ,  1.2284961 , -0.23656592, -0.5976143 ,  1.93649167,
        -0.34299717, -0.4330127 , -0.4330127 , -0.76376262],
       [ 0.        , -0.4024717 ,  0.30592343, -0.5074689 , -0.34299717,
         0.91863042,  0.0281706 ,  0.42130752, -0.5976143 , -0.51639778,
        -0.34299717, -0.43

In [60]:
# Now We Are ready to fit model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [61]:
# Train the model on the scaled training data
model.fit(XtrainScale, y_train)

RandomForestClassifier()

In [62]:
# Make predictions on the scaled test data
predictions = model.predict(XtestScale)

In [63]:
predictions

array([0, 0, 0, 0, 0])

In [64]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [65]:
# Train your model
model.fit(XtrainScale, y_train)

RandomForestClassifier()

In [66]:
# Make predictions on the test set
y_pred = model.predict(XtestScale)

In [67]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)

# Create confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mat)


Accuracy: 1.0
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
Confusion Matrix:
[[5]]


### '''data Is Lill bit coplex, and not in proper our target column has only False values so model will not predict good accuracy

## however i just tried it to make, i did feature engineearing, data correction, vectorize data, pca perfome,etc datat dtypes change train data scale data then apply model'''

# this could be more accurate, but still i din't for best model accuracy perpose ,

# Thank Yo So Much